<a href="https://colab.research.google.com/github/shiva-tech-code/Defect-Detection-in-wall/blob/main/code2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths and categories
DATA_DIR = '/content/drive/My Drive/dp/Wall_Defects_Dataset'
OUTPUT_DIR = '/content/drive/My Drive/Augmented_Dataset'
CATEGORIES = ['cracks', 'chipping', 'stains', 'paint_flaking', 'holes', 'no_defect']

# Create output directory structure
os.makedirs(OUTPUT_DIR, exist_ok=True)
for category in CATEGORIES:
    os.makedirs(os.path.join(OUTPUT_DIR, category), exist_ok=True)

# Augmentation pipeline
augmentation_pipeline = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.MotionBlur(blur_limit=3, p=0.2),
    A.Rotate(limit=15, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
])

# Preprocessing and augmentation
for category in CATEGORIES:
    img_dir = os.path.join(DATA_DIR, category)
    augmented_img_dir = os.path.join(OUTPUT_DIR, category)
    for img_name in tqdm(os.listdir(img_dir), desc=f'Processing {category} images'):
        img_path = os.path.join(img_dir, img_name)
        try:
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError(f"Failed to load {img_name}")
        except Exception as e:
            print(f"Skipping {img_name} due to error: {e}")
            continue

        # Preprocessing
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_img = clahe.apply(gray)
        resized_img = cv2.resize(enhanced_img, (224, 224))
        output_path = os.path.join(augmented_img_dir, f'proc_{img_name}')
        cv2.imwrite(output_path, resized_img)

        # Augmentation
        for i in range(3):
            augmented = augmentation_pipeline(image=image)['image']
            aug_img_path = os.path.join(augmented_img_dir, f'aug_{i}_{img_name}')
            cv2.imwrite(aug_img_path, augmented)

# Create CSV Manifest
image_paths, labels = [], []
for category in CATEGORIES:
    category_dir = os.path.join(OUTPUT_DIR, category)
    for img_name in os.listdir(category_dir):
        img_path = os.path.join(category_dir, img_name)
        image_paths.append(img_path)
        labels.append(category)
df = pd.DataFrame({'image_path': image_paths, 'label': labels})
manifest_path = os.path.join('/content/drive/My Drive', 'dataset_manifest.csv')
df.to_csv(manifest_path, index=False)
print(f"Dataset manifest created as '{manifest_path}'.")


NameError: name 'drive' is not defined

In [ ]:
!pip install tensorflow opencv-python albumentations pandas matplotlib pillow

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm
from google.colab import drive
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
from PIL import Image
import tensorflow as tf

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths and categories
DATA_DIR = '/content/drive/My Drive/dp/Wall_Defects_Dataset'
OUTPUT_DIR = '/content/drive/My Drive/Augmented_Dataset'
CATEGORIES = ['cracks', 'chipping', 'stains', 'paint_flaking', 'holes', 'no_defect']

# Create output directory structure
os.makedirs(OUTPUT_DIR, exist_ok=True)
for category in CATEGORIES:
    os.makedirs(os.path.join(OUTPUT_DIR, category), exist_ok=True)

print("Data directories successfully created.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data directories successfully created.


In [ ]:
# Import libraries
import albumentations as A
import cv2
import pandas as pd
from tqdm import tqdm

# Augmentation pipeline
augmentation_pipeline = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.MotionBlur(blur_limit=3, p=0.2),
    A.Rotate(limit=15, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
])

# Preprocessing and augmentation
for category in CATEGORIES:
    img_dir = os.path.join(DATA_DIR, category)
    augmented_img_dir = os.path.join(OUTPUT_DIR, category)
    for img_name in tqdm(os.listdir(img_dir), desc=f'Processing {category} images'):
        img_path = os.path.join(img_dir, img_name)
        try:
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError(f"Failed to load {img_name}")
        except Exception as e:
            print(f"Skipping {img_name} due to error: {e}")
            continue

        # Preprocessing
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_img = clahe.apply(gray)
        resized_img = cv2.resize(enhanced_img, (224, 224))
        output_path = os.path.join(augmented_img_dir, f'proc_{img_name}')
        cv2.imwrite(output_path, resized_img)

        # Augmentation
        for i in range(3):
            augmented = augmentation_pipeline(image=image)['image']
            aug_img_path = os.path.join(augmented_img_dir, f'aug_{i}_{img_name}')
            cv2.imwrite(aug_img_path, augmented)

# Create CSV Manifest
image_paths, labels = [], []
for category in CATEGORIES:
    category_dir = os.path.join(OUTPUT_DIR, category)
    for img_name in os.listdir(category_dir):
        img_path = os.path.join(category_dir, img_name)
        image_paths.append(img_path)
        labels.append(category)
df = pd.DataFrame({'image_path': image_paths, 'label': labels})
manifest_path = os.path.join('/content/drive/My Drive', 'dataset_manifest.csv')
df.to_csv(manifest_path, index=False)
print(f"Dataset manifest created as '{manifest_path}'.")


Processing no_defect images: 100%|██████████| 50/50 [00:03<00:00, 13.31it/s]


Dataset manifest created as '/content/drive/My Drive/dataset_manifest.csv'.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Loading
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    OUTPUT_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    OUTPUT_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1003 images belonging to 6 classes.
Found 249 images belonging to 6 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:146: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn(


In [ ]:
from tensorflow.keras.applications import MobileNet

def build_mobilenet_model():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(train_generator.num_classes, activation='softmax')
    ])
    return model
